In [6]:
import requests
import pandas as pd
import time

# my personal modules
from wb_scrape_metricCodes import wb_scrape_metricCodes

In [2]:
# !!! If haven't done so, run wb_scrape_metricCodes.py to get the metric codes

# scrape Worldbank's metric codes --> only need to do once
#   will use these to query these metrics from the API

wb_scrape_metricCodes(output_path='output files/wb_metricCodes.csv')

Worldbank metric codes have been exported to csv file.


In [7]:
### INPUTS ###
continent = 'South America'

### MAKE LIST OF COUNTRIES FOR API CALL ###
df = pd.read_csv('input files/wb_countryCodes.csv', skiprows=1) # read CSV that contains Worldbank's country codes for the API URL
country_list = df[(df['Continent'] == continent) & (~df['Country Code'].isna())]['Country Code'].to_list() # filter for only select countries
print(f"country_list: {country_list[:5]}")

# concatenate metrics so that we can bulk get countries via API
country_threshold = 15
country_string_list = []
while country_list: # until it's empty
    country_string = ";".join(country_list[:country_threshold]) # delimit with semi-colon
    country_string_list.append(country_string)
    del country_list[:country_threshold]
print(f"country_string_list: {country_string_list[:2]}")
print(f"len(country_string_list): {len(country_string_list)}")

### MAKE LIST OF METRICS FOR API CALL ###
df = pd.read_csv('wb_metricCodes.csv') # read CSV that contains Worldbank's metric codes for the API URL
df = df.drop(columns=['category', 'name']) # only need the actual code
df = df.drop_duplicates() # remove duplicate metric codes (duplicated on 'category' feature)
metric_list = list(df.squeeze()) # squeeze from df to a series then convert to a list
print(f"metric_list: {metric_list[:5]}")

# concatenate metrics so that we can bulk get metrics via API
metric_threshold = 20
metric_string_list = []
while metric_list: # until it's empty
    metric_string = ";".join(metric_list[:metric_threshold]) # delimit with semi-colon
    metric_string_list.append(metric_string)
    del metric_list[:metric_threshold]
print(f"metric_string_list: {metric_string_list[:2]}")
print(f"len(metric_string_list): {len(metric_string_list)}")

country_list: ['ARG', 'BOL', 'BRA', 'CHL', 'COL']
country_string_list: ['ARG;BOL;BRA;CHL;COL;ECU;GUY;PRY;PER;SUR;URY;VEN']
len(country_string_list): 1
metric_list: ['EG.ELC.ACCS.RU.ZS', 'AG.LND.IRIG.AG.ZS', 'AG.LND.AGRI.ZS', 'AG.LND.AGRI.K2', 'AG.AGR.TRAC.NO']
metric_string_list: ['EG.ELC.ACCS.RU.ZS;AG.LND.IRIG.AG.ZS;AG.LND.AGRI.ZS;AG.LND.AGRI.K2;AG.AGR.TRAC.NO;AG.LND.TRAC.ZS;EN.ATM.METH.AG.ZS;EN.ATM.METH.AG.KT.CE;EN.ATM.NOXE.AG.ZS;EN.ATM.NOXE.AG.KT.CE;TX.VAL.AGRI.ZS.UN;TM.VAL.AGRI.ZS.UN;NV.AGR.TOTL.ZS;NV.AGR.TOTL.CD;ER.H2O.FWAG.ZS;AG.LND.ARBL.ZS;AG.LND.ARBL.HA.PC;AG.LND.ARBL.HA;AG.LND.PRCP.MM;AG.PRD.CREL.MT', 'AG.YLD.CREL.KG;AG.PRD.CROP.XD;SL.AGR.EMPL.ZS;SL.AGR.EMPL.FE.ZS;SL.AGR.EMPL.MA.ZS;AG.CON.FERT.PT.ZS;AG.CON.FERT.ZS;AG.PRD.FOOD.XD;AG.LND.FRST.ZS;AG.LND.FRST.K2;AG.LND.TOTL.K2;AG.LND.CREL.HA;AG.PRD.LVSK.XD;AG.LND.CROP.ZS;AG.LND.TOTL.RU.K2;AG.LND.EL5M.RU.ZS;AG.LND.EL5M.RU.K2;SP.RUR.TOTL;SP.RUR.TOTL.ZS;SP.RUR.TOTL.ZG']
len(metric_string_list): 74


In [14]:
### FROM WORLDBANK GET DATA FOR EACH COUNTRY AND METRIC, OUTPUT A PANDAS.DF ###

# metric_string_list = metric_string_list[:2] # to test the first 2 batches

data_dict = {} # syntax: {(country, date): [x_value, y_value, ..., n_value], (country, date): []}
indicator_list = [] # will use to determine order of values in the 'data_dict'
for x in metric_string_list: # remove concatenation and add to 'indicator_list'
    indicator_list += x.split(';')
indicator_names = [None] * len(indicator_list) # make empty list that can fit each indicator's name

n_metric = 1 # just using this as an incrementor to print out and indicate progress
for x_metrics in metric_string_list:
    print(f"n_metric = {n_metric}")
    country_string = country_string_list[0]
    indicator_string = x_metrics
    url_per_page = 500
    url_page = 1
    r_pages = url_page

    while url_page <= r_pages:
        # info on Worldbank API calls: https://datahelpdesk.worldbank.org/knowledgebase/articles/898581
        # not sure what the 'source' parameter does but when doing multi-indicator calls they only work when 'source=2'
        url = f"https://api.worldbank.org/V2/country/{country_string}/indicator/{indicator_string}?format=json&page={url_page}&per_page={url_per_page}&source=2"
        # 'r' output syntax: [{etc_params}, [{indicator: {etc}, x_1: {etc},.., x_n: {etc}}, ]]
        r = requests.get(url=url).json()
        r_pages = r[0]['pages']
        url_page = url_page + 1
        print(f"url_page: {url_page}, r_pages: {r_pages}")

        for x_record in r[1]: # there's 1 'x_record' per date, country, indicator combo
            x_indicator_name = x_record['indicator']['value'] # can use this to dynamically name the column the indicator value
            x_indicator_code = x_record['indicator']['id']
            x_indicator_index = indicator_list.index(x_indicator_code) # used to determine order of values for the (country, date) key and Pandas.DF columns
            indicator_names[x_indicator_index] = x_indicator_name # add name of column to dynamically use to name Pandas.DF column

            x_value = x_record['value']
            if x_value is None: # if NULL then move on, ensure that the (country, date) combo will only have a record if at least one indicator has a value
                continue
            x_country = x_record['country']['value']
            x_date = x_record['date']
            
            # add value to dictionary
            x_key = (x_country, x_date)
            if x_key not in data_dict.keys():
                data_dict[x_key] = [None] * len(indicator_list) # make empty list that can fit each indicator
            data_dict[x_key][x_indicator_index] = x_value # input value into list
        
        # allow some sleep before next API call to give server a break
        if url_page % 10 == 0:
            time.sleep(2)
    n_metric += 1 

### CONVERT DATA TO FORMAT FOR DF AND CREATE DF ###
# convert data into lists so that it can be easily input into a Pandas DF
data_list = []
for k,v in zip(data_dict.keys(), data_dict.values()):
    x_data = list(k) + v
    data_list.append(x_data)

column_list = ['country', 'year']
column_list += indicator_names
df = pd.DataFrame(data=data_list, columns=column_list)

print(df.shape)
df.head()

In [21]:
# df.to_csv('output files/wb_output.csv', index=False) # export to csv so that we can don't have to scrape anymore

In [18]:
# This is the DF containing the data we pulled from WorldBank API

df = pd.read_csv('output files/wb_output.csv')
df = df.drop(columns=['Unnamed: 0'])
print(df.shape)
print(df.dtypes)
df.head()

(756, 1474)
country                                                                             object
year                                                                                 int64
Access to electricity, rural (% of rural population)                               float64
Agricultural irrigated land (% of total agricultural land)                         float64
Agricultural land (% of land area)                                                 float64
                                                                                    ...   
Wage and salaried workers, total (% of total employment) (modeled ILO estimate)    float64
Population density (people per sq. km of land area)                                float64
Population in largest city                                                         float64
Population in the largest city (% of urban population)                             float64
Population in urban agglomerations of more than 1 million                     

,country,year,"Access to electricity, rural (% of rural population)",Agricultural irrigated land (% of total agricultural land),Agricultural land (% of land area),Agricultural land (sq. km),"Agricultural machinery, tractors","Agricultural machinery, tractors per 100 sq. km of arable land",Agricultural methane emissions (% of total),Agricultural methane emissions (thousand metric tons of CO2 equivalent),...,"Unemployment, total (% of total labor force) (national estimate)","Unemployment, youth female (% of female labor force ages 15-24) (national estimate)","Unemployment, youth male (% of male labor force ages 15-24) (national estimate)","Unemployment, youth total (% of total labor force ages 15-24) (modeled ILO estimate)","Unemployment, youth total (% of total labor force ages 15-24) (national estimate)","Wage and salaried workers, total (% of total employment) (modeled ILO estimate)",Population density (people per sq. km of land area),Population in largest city,Population in the largest city (% of urban population),Population in urban agglomerations of more than 1 million
0,Argentina,2021,100.000000,NaN,43.102346,1179577.597,NaN,NaN,NaN,NaN,...,8.74,26.68,20.77,23.370,23.24,71.39813,16.738742,15257673.0,36.113734,19587272.0
1,Argentina,2020,100.000000,NaN,43.029265,1177577.597,NaN,NaN,NaN,86098.7125,...,11.46,34.95,26.81,30.407,30.17,73.00002,16.580893,15153729.0,36.255551,19430420.0
2,Argentina,2019,100.000000,NaN,42.590779,1165577.597,NaN,NaN,NaN,85762.3750,...,9.84,28.78,23.87,25.812,25.86,73.49278,16.420827,15057273.0,36.423390,19280518.0
3,Argentina,2018,99.922955,1.199575,42.371536,1159577.597,NaN,NaN,NaN,85839.3425,...,9.22,27.77,20.85,23.830,23.73,74.47115,16.258510,14966530.0,36.613484,19136241.0
4,Argentina,2017,100.000000,NaN,42.986381,1176404.000,NaN,NaN,NaN,86592.7575,...,8.35,26.90,19.86,22.831,22.64,74.66000,16.094191,14879100.0,36.819727,18995646.0


In [4]:
print(df.count())

country                                                                            756
year                                                                               756
Access to electricity, rural (% of rural population)                               354
Agricultural irrigated land (% of total agricultural land)                          54
Agricultural land (% of land area)                                                 732
                                                                                  ... 
Wage and salaried workers, total (% of total employment) (modeled ILO estimate)    372
Population density (people per sq. km of land area)                                732
Population in largest city                                                         630
Population in the largest city (% of urban population)                             630
Population in urban agglomerations of more than 1 million                          630
Length: 1474, dtype: int64
